# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [27]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///Massages.db')
df = pd.read_sql("clean_msg", engine)
X = df['message']
y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def tokenize(text):
    """
    Description: 
        For a dataframe with massages, do following procedures.
        - Normalization to lowercase.
        - Remove punctuation characters.
        - Tokenization,lemmatization, and stop word removal.
        
    Arguments:
        text: text
    Returns:
        tokens: a list of tockenized text.
    """
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('forest', MultiOutputClassifier( RandomForestClassifier() ))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =123, test_size =0.2)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [6]:
y_train.columns[0]

'related'

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
y_pred_t = np.transpose(y_pred)
y_pred_t.shape

(36, 5236)

In [9]:
i = 0
while i<len(y_pred_t):
    print(y_train.columns[i])
    print(classification_report(y_test[y_train.columns[i]], y_pred_t[i]))
    i = i+1

related
             precision    recall  f1-score   support

          0       0.62      0.48      0.54      1244
          1       0.85      0.90      0.87      3957
          2       0.24      0.31      0.27        35

avg / total       0.79      0.80      0.79      5236

request
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      4334
          1       0.78      0.45      0.57       902

avg / total       0.87      0.88      0.87      5236

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5212
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5236

aid_related
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      3144
          1       0.75      0.59      0.66      2092

avg / total       0.76      0.76      0.75      5236

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
parameters = {
    'tfidf__norm': ['l1', 'l2'],
    'tfidf__sublinear_tf': [True, False] 
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=3, n_jobs=-1)

In [21]:
cv.fit(X_train, y_train)
cv.cv_results_ 

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=True, score=0.2257556224036671, total=  53.9s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  tfidf__norm=l1, tfidf__sublinear_tf=True, score=0.23868194842406876, total=  53.9s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


[CV]  tfidf__norm=l1, tfidf__sublinear_tf=True, score=0.23581661891117478, total=  53.8s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=False .......................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=False, score=0.23449362555507808, total=  53.7s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=False .......................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=False, score=0.24025787965616047, total=  53.5s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=False .......................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=False, score=0.23696275071633238, total=  54.3s
[CV] tfidf__norm=l2, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l2, tfidf__sublinear_tf=True, score=0.2301962469560235, total=  53.6s
[CV] tfidf__norm=l2, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l2, tfidf__sublinear_tf=True, score=0.23796561604584526, total=  53.7s
[CV] tfidf__norm=l2, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l2, tfidf__sublinear

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 12.8min finished
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwar

{'mean_fit_time': array([ 48.83704559,  48.7476608 ,  48.62592491,  48.85132917]),
 'std_fit_time': array([ 0.06945473,  0.33246915,  0.17131365,  0.23846904]),
 'mean_score_time': array([ 5.05005542,  5.07015324,  5.10583901,  5.09127951]),
 'std_score_time': array([ 0.07133562,  0.05722619,  0.05644178,  0.0429946 ]),
 'param_tfidf__norm': masked_array(data = ['l1' 'l1' 'l2' 'l2'],
              mask = [False False False False],
        fill_value = ?),
 'param_tfidf__sublinear_tf': masked_array(data = [True False True False],
              mask = [False False False False],
        fill_value = ?),
 'params': [{'tfidf__norm': 'l1', 'tfidf__sublinear_tf': True},
  {'tfidf__norm': 'l1', 'tfidf__sublinear_tf': False},
  {'tfidf__norm': 'l2', 'tfidf__sublinear_tf': True},
  {'tfidf__norm': 'l2', 'tfidf__sublinear_tf': False}],
 'split0_test_score': array([ 0.22575562,  0.23449363,  0.23019625,  0.22919353]),
 'split1_test_score': array([ 0.23868195,  0.24025788,  0.23796562,  0.23166189]

In [23]:
y_pred2 = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
y_pred_t2 = np.transpose(y_pred2)
y_pred_t2.shape

(36, 5236)

In [25]:
i = 0
while i<len(y_pred_t2):
    print(y_train.columns[i])
    print(classification_report(y_test[y_train.columns[i]], y_pred_t2[i]))
    i = i+1

related
             precision    recall  f1-score   support

          0       0.64      0.48      0.54      1244
          1       0.84      0.92      0.88      3957
          2       0.58      0.20      0.30        35

avg / total       0.79      0.81      0.79      5236

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4334
          1       0.80      0.44      0.57       902

avg / total       0.88      0.88      0.87      5236

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5212
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5236

aid_related
             precision    recall  f1-score   support

          0       0.76      0.85      0.80      3144
          1       0.73      0.61      0.66      2092

avg / total       0.75      0.75      0.75      5236

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [31]:
with open("fitted_model.pickle", 'wb') as pickle_file:
    pickle.dump(cv, pickle_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.